In [30]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from scipy.spatial.distance import cosine

embeddings = HuggingFaceEmbeddings()

text = "This is a test document."

query_result = embeddings.embed_query(text)

len(query_result)


# calculate cosine similarity between string1 and string2


string1 = "This is a test document"
string2 = "This is a long paragraph"

embed1 = embeddings.embed_query(string1)
embed2 = embeddings.embed_query(string2)

cosine(embed1, embed2)

print(len(embed1))

# Now we will use pinecone in langchain




768


In [49]:
import pinecone   
from langchain_community.vectorstores import Pinecone   
import numpy as np

pinecone.init(      
	api_key='2f9572cf-4cf8-49fc-b48e-4f7d4fc6021f',      
	environment='gcp-starter'      
)      
index_name = 'find-documents'
index = pinecone.Index(index_name)
embeddings = HuggingFaceEmbeddings()

if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric='cosine', shards=1, dimension=768)
    
vector_store = Pinecone(index, embeddings,  "text")

key = 'aaa'
text = "this is a test document number two"
vector_store.add_texts([text], namespace=key)

results = vector_store.similarity_search_with_score("two documents", k=2, ) #namespace=key)
print(results)





[(Document(page_content='this is a test document'), 0.459236085)]


In [34]:
import pinecone
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Pinecone
from nltk.tokenize import word_tokenize, sent_tokenize
from itertools import islice, chain
import nltk
import string
import time
#nltk.download('punkt')

# Initialize Pinecone
pinecone.init(      
	api_key='2f9572cf-4cf8-49fc-b48e-4f7d4fc6021f',      
	environment='gcp-starter'      
)   
index_name = 'find-documents'

# Check if index exists, if not create one
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric='cosine', shards=1, dimension=384)

# Create an index and embeddings
index = pinecone.Index(index_name)
embeddings = FastEmbedEmbeddings()
vector_store = Pinecone(index, embeddings, "text")

def ngrams_with_index(input_string, n):
    # Join the words into a single string
    joined_string = " ".join(input_string)

    # Create n-grams along with start and end character indices
    ngrams_with_indices = []
    words = joined_string.split()  # Split into words to create n-grams
    current_index = 0  # Tracks the current index in the joined_string

    for i in range(len(words) - n + 1):
        # Create the n-gram
        ngram = ' '.join(words[i:i + n])

        # Calculate the start index based on the current index
        start_idx = current_index

        # Calculate the end index
        end_idx = start_idx + len(ngram) - 1

        # Update current_index for the next iteration
        # we only shift one word at a time
        current_index += len(words[i]) + 1
        ngrams_with_indices.append((ngram, start_idx, end_idx))

    return ngrams_with_indices

def remove_punctuation(text):
    return ''.join(char for char in text if char not in string.punctuation)

# Process the document
def process_document(document, key):
    # Tokenize the document
    unigrams = ngrams_with_index(word_tokenize(document), 1)
    bigrams = ngrams_with_index(word_tokenize(document), 2)
    trigrams = ngrams_with_index(word_tokenize(document), 3)

    text_list = []
    metadata_list = []
    # Add all n-grams to pinecone
    for unigram, start, end in unigrams:
        text = remove_punctuation(unigram)
        if text == '':
            continue
        text_list.append(text)
        metadata_list.append({'start': start, 'end': end})
        #vector_store.add_texts([text], namespace=key, metadatas=[{'start': start, 'end': end}])

    for bigram, start, end in bigrams:
        text = remove_punctuation(bigram)
        if len(text.split()) < 2:
            continue
        text_list.append(text)
        metadata_list.append({'start': start, 'end': end})
        #vector_store.add_texts([text], namespace=key, metadatas=[{'start': start, 'end': end}])

    for trigram, start, end in trigrams:
        text = remove_punctuation(trigram)
        if len(text.split()) < 3:
            continue
        text_list.append(text)
        metadata_list.append({'start': start, 'end': end})
        #vector_store.add_texts([text], namespace=key, metadatas=[{'start': start, 'end': end}])
        
    #vector_store.add_texts(text_list, namespace=key, batch_size=500)
    # convert text_list embeddings
    vector_store.add_texts(text_list, namespace=key, batch_size=100, embedding_chunk_size=5000, metadatas=metadata_list)

# Example usage
import requests
document = "This is a test document. It has several sentences, and it's a good example for n-gram processing."*250

start = time.time()
process_document(document, "testv1")
end = time.time()
print(end - start)


99.33884572982788


In [30]:
x = vector_store.similarity_search_with_score("test", k=2, namespace="D1JFQ80WXFYAJLJD4ZI1QKQSWBO0DDBG")

In [21]:
!pip3 install fastembed

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 6.2 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 16.3 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 394 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 11.2 MB/s eta 0:00:01
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
     |████████████████████████████████| 86 kB 5.5 MB/s eta 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.2
    Uninstalling huggingface-hub-0.20.2:
      Successfully uninstalled huggingface-hub-0.20.2
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [17]:
vector_store.delete(delete_all=True, namespace="testv1")

In [34]:
help(Pinecone)

Help on class Pinecone in module langchain_community.vectorstores.pinecone:

class Pinecone(langchain_core.vectorstores.VectorStore)
 |  Pinecone(index: 'Any', embedding: 'Union[Embeddings, Callable]', text_key: 'str', namespace: 'Optional[str]' = None, distance_strategy: 'Optional[DistanceStrategy]' = <DistanceStrategy.COSINE: 'COSINE'>)
 |  
 |  `Pinecone` vector store.
 |  
 |  To use, you should have the ``pinecone-client`` python package installed.
 |  
 |  Example:
 |      .. code-block:: python
 |  
 |          from langchain_community.vectorstores import Pinecone
 |          from langchain_community.embeddings.openai import OpenAIEmbeddings
 |          import pinecone
 |  
 |          # The environment should be the one specified next to the API key
 |          # in your Pinecone console
 |          pinecone.init(api_key="***", environment="...")
 |          index = pinecone.Index("langchain-demo")
 |          embeddings = OpenAIEmbeddings()
 |          vectorstore = Pinecone(i

In [52]:
!pip3 install langchain
!pip3 install langchain sentence_transformers
!pip3 install pinecone-client
!pip3 install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import pinecone
from langchain_community.embeddings import HuggingFaceEmbeddings, SentenceTransformersEmbeddings
from langchain_community.vectorstores import Pinecone
from nltk.tokenize import word_tokenize, sent_tokenize
from itertools import islice, chain
import nltk
import string
import time
#nltk.download('punkt')

# Initialize Pinecone
pinecone.init(      
	api_key='2f9572cf-4cf8-49fc-b48e-4f7d4fc6021f',      
	environment='gcp-starter'      
)   
index_name = 'find-documents'

# Check if index exists, if not create one
if index_name not in pinecone.list_indexes():
    pinecone.create_index(name=index_name, metric='cosine', shards=1, dimension=768)

# Create an index and embeddings
index = pinecone.Index(index_name)
embeddings = SentenceTransformersEmbeddings()
vector_store = Pinecone(index, embeddings, "text")

def ngrams_with_index(input_string, n):
    # Join the words into a single string
    joined_string = " ".join(input_string)

    # Create n-grams along with start and end character indices
    ngrams_with_indices = []
    words = joined_string.split()  # Split into words to create n-grams
    current_index = 0  # Tracks the current index in the joined_string

    for i in range(len(words) - n + 1):
        # Create the n-gram
        ngram = ' '.join(words[i:i + n])

        # Calculate the start index based on the current index
        start_idx = current_index

        # Calculate the end index
        end_idx = start_idx + len(ngram) - 1

        # Update current_index for the next iteration
        # we only shift one word at a time
        current_index += len(words[i]) + 1
        ngrams_with_indices.append((ngram, start_idx, end_idx))

    return ngrams_with_indices

def remove_punctuation(text):
    return ''.join(char for char in text if char not in string.punctuation)

# Process the document
def process_document(document, key):
    # Tokenize the document
    unigrams = ngrams_with_index(word_tokenize(document), 1)
    sentence_embeddings = embeddings.embed_documents(sent_tokenize(document))
    
  
    #vector_store.add_texts([text], namespace=key, metadatas=[{'start': start, 'end': end}])
        

# Example usage
import requests
document = "This is a test document. It has several sentences, and it's a good example for n-gram processing."*100
#document = requests.get("https://www.gutenberg.org/cache/epub/35/pg35.txt").text


start = time.time()
process_document(document, "testv1")
end = time.time()
print(end - start)
